In [ ]:
# Configuración del entorno y librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.2f}'.format)

print("📊 Entorno configurado exitosamente")
print(f"📁 Directorio de trabajo: {Path.cwd()}")


In [ ]:
# Funciones helper para análisis

def format_currency(value):
    """Formatear valores monetarios"""
    if pd.isna(value):
        return 'N/A'
    if value >= 1e9:
        return f"${value/1e9:.2f}B"
    elif value >= 1e6:
        return f"${value/1e6:.2f}M"
    elif value >= 1e3:
        return f"${value/1e3:.2f}K"
    else:
        return f"${value:.2f}"

def format_number(value):
    """Formatear números grandes"""
    if pd.isna(value):
        return 'N/A'
    if value >= 1e6:
        return f"{value/1e6:.1f}M"
    elif value >= 1e3:
        return f"{value/1e3:.1f}K"
    else:
        return f"{value:,.0f}"

def print_metric(label, value, is_currency=False):
    """Imprimir métrica formateada"""
    if is_currency:
        formatted_value = format_currency(value)
    else:
        formatted_value = format_number(value)
    print(f"📈 {label}: {formatted_value}")

print("🔧 Funciones helper cargadas")


In [ ]:
# Carga y validación de datos

try:
    # Cargar datos procesados
    data_path = Path('data/processed/standardized_complete_data.csv')
    
    if not data_path.exists():
        print(f"❌ Archivo no encontrado: {data_path}")
        print("💡 Ejecuta primero el script de procesamiento: python scripts/run_complete_analysis.py")
        raise FileNotFoundError(f"No se encontró {data_path}")
    
    print(f"📂 Cargando datos desde: {data_path}")
    df = pd.read_csv(data_path)
    
    print(f"✅ Datos cargados exitosamente")
    print(f"📊 Dimensiones del dataset: {df.shape[0]:,} filas x {df.shape[1]} columnas")
    
    # Validación básica
    required_columns = ['patient_id', 'total_cost', 'units', 'origin']
    missing_cols = [col for col in required_columns if col not in df.columns]
    
    if missing_cols:
        print(f"⚠️  Columnas faltantes: {missing_cols}")
    else:
        print("✅ Todas las columnas requeridas están presentes")
        
except Exception as e:
    print(f"❌ Error al cargar datos: {e}")
    raise


In [ ]:
# Vista general del dataset

print("="*60)
print("📋 RESUMEN GENERAL DEL DATASET")
print("="*60)

# Información básica
print(f"\n🔍 INFORMACIÓN BÁSICA:")
print_metric("Total de registros (cargos)", len(df))
print_metric("Pacientes únicos", df['patient_id'].nunique())
print_metric("Rango de fechas", f"{df['date'].min()} a {df['date'].max()}")
print_metric("Orígenes de servicio", df['origin'].nunique())

# Métricas financieras principales
print(f"\n💰 MÉTRICAS FINANCIERAS:")
print_metric("Costo total acumulado", df['total_cost'].sum(), is_currency=True)
print_metric("Unidades totales facturadas", df['units'].sum())
print_metric("Costo promedio por cargo", df['total_cost'].mean(), is_currency=True)
print_metric("Costo promedio por unidad", (df['total_cost'].sum() / df['units'].sum()), is_currency=True)

# Estadísticas de distribución
print(f"\n📊 DISTRIBUCIÓN DE COSTOS:")
print_metric("Costo mínimo por cargo", df['total_cost'].min(), is_currency=True)
print_metric("Costo máximo por cargo", df['total_cost'].max(), is_currency=True)
print_metric("Mediana de costos", df['total_cost'].median(), is_currency=True)
print_metric("Desviación estándar", df['total_cost'].std(), is_currency=True)

print("\n✅ Resumen general completado")


In [ ]:
# Análisis por origen de servicio

print("="*60)
print("🏥 ANÁLISIS POR ORIGEN DE SERVICIO")
print("="*60)

# Agrupación por origen
origin_analysis = df.groupby('origin').agg({
    'patient_id': 'count',  # número de cargos
    'total_cost': ['sum', 'mean', 'median'],
    'units': ['sum', 'mean'],
}).round(2)

# Simplificar nombres de columnas
origin_analysis.columns = ['total_cargos', 'costo_total', 'costo_promedio', 'costo_mediano', 
                          'unidades_total', 'unidades_promedio']

# Calcular pacientes únicos por origen
pacientes_por_origen = df.groupby('origin')['patient_id'].nunique()
origin_analysis['pacientes_unicos'] = pacientes_por_origen

# Calcular métricas adicionales
origin_analysis['costo_por_unidad'] = origin_analysis['costo_total'] / origin_analysis['unidades_total']
origin_analysis['pct_cargos'] = (origin_analysis['total_cargos'] / origin_analysis['total_cargos'].sum() * 100).round(1)
origin_analysis['pct_costo'] = (origin_analysis['costo_total'] / origin_analysis['costo_total'].sum() * 100).round(1)

print("\n📊 RESUMEN POR ORIGEN:")
for origin in origin_analysis.index:
    row = origin_analysis.loc[origin]
    print(f"\n🔸 {origin.upper()}:")
    print(f"   • Cargos: {format_number(row['total_cargos'])} ({row['pct_cargos']}% del total)")
    print(f"   • Pacientes únicos: {format_number(row['pacientes_unicos'])}")
    print(f"   • Costo total: {format_currency(row['costo_total'])} ({row['pct_costo']}% del total)")
    print(f"   • Costo por unidad: {format_currency(row['costo_por_unidad'])}")
    print(f"   • Unidades totales: {format_number(row['unidades_total'])}")

print("\n✅ Análisis por origen completado")


In [ ]:
# Análisis de pacientes por complejidad

print("="*60)
print("👥 ANÁLISIS DE PACIENTES POR COMPLEJIDAD")
print("="*60)

# Análisis por paciente
patient_analysis = df.groupby('patient_id').agg({
    'total_cost': 'sum',
    'units': 'sum',
    'patient_id': 'count',  # número de cargos por paciente
    'origin': lambda x: list(x.unique())  # orígenes únicos por paciente
}).rename(columns={'patient_id': 'num_cargos'})

# Calcular métricas adicionales
patient_analysis['costo_por_cargo'] = patient_analysis['total_cost'] / patient_analysis['num_cargos']
patient_analysis['num_origenes'] = patient_analysis['origin'].apply(len)

# Categorización por complejidad (número de cargos)
def categorize_complexity(num_cargos):
    if num_cargos == 1:
        return 'Unitario'
    elif num_cargos <= 5:
        return 'Baja complejidad'
    elif num_cargos <= 20:
        return 'Media complejidad'
    elif num_cargos <= 100:
        return 'Alta complejidad'
    else:
        return 'Extrema complejidad'

patient_analysis['categoria_complejidad'] = patient_analysis['num_cargos'].apply(categorize_complexity)

# Análisis por categoría
complexity_analysis = patient_analysis.groupby('categoria_complejidad').agg({
    'total_cost': ['count', 'sum', 'mean'],
    'num_cargos': ['sum', 'mean'],
    'units': ['sum', 'mean']
}).round(2)

complexity_analysis.columns = ['num_pacientes', 'costo_total', 'costo_promedio',
                              'cargos_total', 'cargos_promedio', 
                              'unidades_total', 'unidades_promedio']

# Ordenar por complejidad
order = ['Unitario', 'Baja complejidad', 'Media complejidad', 'Alta complejidad', 'Extrema complejidad']
complexity_analysis = complexity_analysis.reindex([cat for cat in order if cat in complexity_analysis.index])

print("\n📊 ANÁLISIS POR COMPLEJIDAD DE PACIENTES:")
for category in complexity_analysis.index:
    row = complexity_analysis.loc[category]
    pct_pacientes = (row['num_pacientes'] / complexity_analysis['num_pacientes'].sum() * 100)
    pct_costos = (row['costo_total'] / complexity_analysis['costo_total'].sum() * 100)
    
    print(f"\n🔸 {category.upper()}:")
    print(f"   • Pacientes: {format_number(row['num_pacientes'])} ({pct_pacientes:.1f}% del total)")
    print(f"   • Costo total: {format_currency(row['costo_total'])} ({pct_costos:.1f}% del total)")
    print(f"   • Costo promedio por paciente: {format_currency(row['costo_promedio'])}")
    print(f"   • Cargos promedio por paciente: {row['cargos_promedio']:.1f}")

print("\n✅ Análisis de complejidad completado")


In [ ]:
# Visualización: Distribución por origen y complejidad

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Análisis de Distribución - Origen y Complejidad', fontsize=16, fontweight='bold')

# 1. Distribución de cargos por origen
origin_analysis['pct_cargos'].plot(kind='bar', ax=axes[0,0], color='skyblue')
axes[0,0].set_title('Distribución de Cargos por Origen (%)')
axes[0,0].set_ylabel('Porcentaje')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Costo por unidad por origen
origin_analysis['costo_por_unidad'].plot(kind='bar', ax=axes[0,1], color='lightgreen')
axes[0,1].set_title('Costo por Unidad por Origen')
axes[0,1].set_ylabel('Costo ($)')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Distribución de pacientes por complejidad
complexity_pct = (complexity_analysis['num_pacientes'] / complexity_analysis['num_pacientes'].sum() * 100)
complexity_pct.plot(kind='bar', ax=axes[1,0], color='orange')
axes[1,0].set_title('Distribución de Pacientes por Complejidad (%)')
axes[1,0].set_ylabel('Porcentaje')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Costo promedio por categoría de complejidad
complexity_analysis['costo_promedio'].plot(kind='bar', ax=axes[1,1], color='pink')
axes[1,1].set_title('Costo Promedio por Categoría de Complejidad')
axes[1,1].set_ylabel('Costo ($)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("📈 Gráficos de distribución generados")


In [ ]:
# Top 10 pacientes con mayor costo

print("="*60)
print("🏆 TOP 10 PACIENTES POR COSTO TOTAL")
print("="*60)

top_patients = patient_analysis.nlargest(10, 'total_cost')

print(f"\n📊 RANKING DE PACIENTES MÁS COSTOSOS:")
for i, (patient_id, row) in enumerate(top_patients.iterrows(), 1):
    print(f"\n🏅 #{i} - Paciente {patient_id}:")
    print(f"   • Costo total: {format_currency(row['total_cost'])}")
    print(f"   • Número de cargos: {row['num_cargos']:,}")
    print(f"   • Costo por cargo: {format_currency(row['costo_por_cargo'])}")
    print(f"   • Unidades totales: {format_number(row['units'])}")
    print(f"   • Orígenes de servicio: {', '.join(row['origin'])}")
    print(f"   • Categoría: {row['categoria_complejidad']}")

# Impacto del top 10
top10_cost = top_patients['total_cost'].sum()
total_cost = patient_analysis['total_cost'].sum()
pct_impact = (top10_cost / total_cost * 100)

print(f"\n💡 IMPACTO DEL TOP 10:")
print_metric("Costo combinado top 10", top10_cost, is_currency=True)
print_metric("Porcentaje del costo total", f"{pct_impact:.1f}%")
print_metric("Costo promedio top 10", top_patients['total_cost'].mean(), is_currency=True)

print("\n✅ Análisis de top pacientes completado")


In [ ]:
# Análisis de eficiencia: relación cantidad vs costo

print("="*60)
print("⚡ ANÁLISIS DE EFICIENCIA: CANTIDAD vs COSTO")
print("="*60)

# Calcular costo por unidad para cada cargo
df['costo_por_unidad'] = df['total_cost'] / df['units']

# Filtrar valores infinitos o NaN
df_clean = df[np.isfinite(df['costo_por_unidad']) & (df['units'] > 0)].copy()

print(f"\n📊 MÉTRICAS DE EFICIENCIA GENERAL:")
print_metric("Costo por unidad promedio", df_clean['costo_por_unidad'].mean(), is_currency=True)
print_metric("Costo por unidad mediano", df_clean['costo_por_unidad'].median(), is_currency=True)
print_metric("Costo por unidad mínimo", df_clean['costo_por_unidad'].min(), is_currency=True)
print_metric("Costo por unidad máximo", df_clean['costo_por_unidad'].max(), is_currency=True)

# Coeficiente de variación
cv = (df_clean['costo_por_unidad'].std() / df_clean['costo_por_unidad'].mean()) * 100
print_metric("Coeficiente de variación", f"{cv:.0f}%")

# Análisis de eficiencia por origen
print(f"\n🔍 EFICIENCIA POR ORIGEN DE SERVICIO:")
efficiency_by_origin = df_clean.groupby('origin')['costo_por_unidad'].agg([
    'mean', 'median', 'std', 'min', 'max', 'count'
]).round(2)

for origin in efficiency_by_origin.index:
    row = efficiency_by_origin.loc[origin]
    cv_origin = (row['std'] / row['mean']) * 100 if row['mean'] > 0 else 0
    
    print(f"\n🔸 {origin.upper()}:")
    print(f"   • Costo/unidad promedio: {format_currency(row['mean'])}")
    print(f"   • Costo/unidad mediano: {format_currency(row['median'])}")
    print(f"   • Variabilidad (CV): {cv_origin:.0f}%")
    print(f"   • Rango: {format_currency(row['min'])} - {format_currency(row['max'])}")
    print(f"   • Número de cargos: {format_number(row['count'])}")

print("\n✅ Análisis de eficiencia completado")


# 📊 **Análisis Exploratorio de Datos Hospitalarios**
## **Vista General del Dataset Estandarizado**

---

<div style="background-color: #f8f9fa; padding: 20px; border-left: 5px solid #007bff; margin: 20px 0;">

### 📋 **Información del Documento**

| **Campo** | **Valor** |
|-----------|-----------|
| **Proyecto** | Economía de la Salud - UCDS INER |
| **Fecha** | 7 de Enero, 2025 |
| **Autor** | Equipo de Ciencia de Datos |
| **Dataset** | `resultados_pacientes_estandarizados.csv` |
| **Versión** | 1.2 - Análisis Profesional |

</div>

---

### 🎯 **Objetivos del Análisis**

1. **📈 Caracterización completa** del dataset hospitalario estandarizado
2. **🔍 Validación de insights** sobre flujos de atención médica  
3. **💰 Análisis financiero** detallado por servicios y pacientes
4. **📊 Identificación de patrones** en la utilización de servicios
5. **🚀 Preparación de métricas** para dashboard ejecutivo

---

### 📊 **Resumen Ejecutivo de Hallazgos**

<div style="background-color: #e8f5e8; padding: 15px; border-radius: 8px; margin: 15px 0;">

#### **✅ Métricas Principales**
- **2.4M cargos hospitalarios** de **5,782 pacientes únicos**
- **13.2M unidades** de servicios facturadas
- **$1.18B** en costos totales (período 2024-2025)
- **99.98%** de cargos siguen flujo Triage → Hospitalización

</div>

---


In [1]:
# =========================================================================
# CONFIGURACIÓN DEL ENTORNO Y LIBRERÍAS
# =========================================================================

# Librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime

# Configuración general
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_rows', 100)

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Colores corporativos
COLORS = {
    'primary': '#007bff',
    'secondary': '#6c757d', 
    'success': '#28a745',
    'danger': '#dc3545',
    'warning': '#ffc107',
    'info': '#17a2b8'
}

# Función para formatear números
def format_number(num):
    """Formatea números grandes con comas y sufijos apropiados"""
    if num >= 1_000_000:
        return f"{num/1_000_000:.1f}M"
    elif num >= 1_000:
        return f"{num/1_000:.0f}K"
    else:
        return f"{num:,.0f}"

def format_currency(amount):
    """Formatea montos como moneda"""
    if amount >= 1_000_000_000:
        return f"${amount/1_000_000_000:.2f}B"
    elif amount >= 1_000_000:
        return f"${amount/1_000_000:.1f}M"
    elif amount >= 1_000:
        return f"${amount/1_000:.0f}K"
    else:
        return f"${amount:,.2f}"

print("✅ Configuración completada")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 Numpy version: {np.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn version: {sns.__version__}")
print(f"⏰ Análisis iniciado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


✅ Configuración completada
📊 Pandas version: 2.3.0
🔢 Numpy version: 1.26.4
📈 Matplotlib version: 3.10.3
🎨 Seaborn version: 0.13.2
⏰ Análisis iniciado: 2025-07-07 12:14:19


In [2]:
# =========================================================================
# CARGA Y VALIDACIÓN INICIAL DE DATOS
# =========================================================================

print("="*80)
print("📂 CARGANDO DATASET HOSPITALARIO ESTANDARIZADO")
print("="*80)

# Cargar dataset con configuración optimizada
try:
    df = pd.read_csv('../../data/processed/resultados_pacientes_estandarizados.csv', 
                     low_memory=False)
    print("✅ Dataset cargado exitosamente")
except FileNotFoundError:
    print("❌ Error: No se pudo encontrar el archivo de datos")
    raise
except Exception as e:
    print(f"❌ Error inesperado: {e}")
    raise

# Información básica del dataset
print(f"\n📏 **DIMENSIONES DEL DATASET**")
print(f"   • Filas: {df.shape[0]:,}")
print(f"   • Columnas: {df.shape[1]}")
print(f"   • Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Mostrar estructura de columnas
print(f"\n📋 **ESTRUCTURA DE COLUMNAS** ({len(df.columns)} total)")
print("-" * 60)
for i, col in enumerate(df.columns, 1):
    dtype = str(df[col].dtype)
    null_count = df[col].isnull().sum()
    unique_count = df[col].nunique()
    print(f"   {i:2d}. {col:<25} | {dtype:<10} | {null_count:>8,} nulls | {unique_count:>8,} únicos")

# Vista previa de los datos
print(f"\n👀 **VISTA PREVIA DE LOS DATOS**")
print("-" * 60)
display(df.head(3))


📂 CARGANDO DATASET HOSPITALARIO ESTANDARIZADO
✅ Dataset cargado exitosamente

📏 **DIMENSIONES DEL DATASET**
   • Filas: 2,399,200
   • Columnas: 18
   • Tamaño en memoria: 2068.9 MB

📋 **ESTRUCTURA DE COLUMNAS** (18 total)
------------------------------------------------------------
    1. paciente                  | int64      |        0 nulls |    5,782 únicos
    2. id_registro_admision      | float64    |  551,420 nulls |    6,617 únicos
    3. id_registro_urg           | float64    | 1,847,779 nulls |    5,348 únicos
    4. origen                    | object     |        0 nulls |        3 únicos
    5. fecha                     | object     |        0 nulls |  533,766 únicos
    6. cantidad                  | float64    |        0 nulls |      310 únicos
    7. clave                     | object     |        0 nulls |    1,926 únicos
    8. descripcion               | object     |        0 nulls |    1,916 únicos
    9. area_servicio             | object     |   21,408 nulls |   

,paciente,id_registro_admision,id_registro_urg,origen,fecha,cantidad,clave,descripcion,area_servicio,nivel,costo_nivel_6,monto_nivel_1,monto_nivel_6,n_expediente_hosp,ian_expediente_hosp,fecha_egreso_general,archivo_origen,n_expediente_hosp_original
0,627611,NaN,35448.00,Urgencias,2023-12-27 08:47:35.000,1.00,UR102,Valoración en consultorio de urgencias,URGENCIAS,2E,60.00,60.00,60.00,000255403,IAN714865,2024-01-01 12:39:22.000,Resultados Pacientes Jan 2024 - Jul 2024.csv,255403
1,627611,NaN,35448.00,Laboratorios,2023-12-27 08:49:10.000,1.00,LC126,Procalcitonina,Laboratorios Clínicos,2E,767.00,77.00,767.00,000255403,IAN714865,2024-01-01 12:39:22.000,Resultados Pacientes Jan 2024 - Jul 2024.csv,255403
2,627611,NaN,35448.00,Laboratorios,2023-12-27 08:49:10.000,1.00,RA70,Tórax lateral (1 imagen),Radiología,2E,170.00,46.00,170.00,000255403,IAN714865,2024-01-01 12:39:22.000,Resultados Pacientes Jan 2024 - Jul 2024.csv,255403


---

## 📊 **1. ESTADÍSTICAS DESCRIPTIVAS GENERALES**

En esta sección analizamos las **características fundamentales** del dataset hospitalario, incluyendo distribuciones de pacientes, cargos, unidades de servicio y métricas financieras.

<div style="background-color: #fff3cd; padding: 15px; border-left: 4px solid #ffc107; margin: 15px 0;">
    <strong>💡 Conceptos Clave:</strong><br>
    • <strong>Cargo:</strong> Registro individual de facturación hospitalaria<br>
    • <strong>Unidad:</strong> Cantidad específica de servicios/productos dentro de un cargo<br>
    • <strong>Costo por Unidad:</strong> Precio unitario promedio de cada servicio
</div>


In [3]:
# =========================================================================
# MÉTRICAS PRINCIPALES DEL DATASET
# =========================================================================

print("="*80)
print("📈 ESTADÍSTICAS DESCRIPTIVAS PRINCIPALES")
print("="*80)

# Calcular métricas principales
total_cargos = len(df)
pacientes_unicos = df['paciente'].nunique()
expedientes_unicos = df['n_expediente_hosp'].nunique()
ian_unicos = df['ian_expediente_hosp'].nunique()
origenes_unicos = df['origen'].nunique()
areas_servicio_unicos = df['area_servicio'].nunique()
total_unidades = df['cantidad'].sum()
total_costo = df['monto_nivel_6'].sum()

# Crear tabla de métricas principales
metricas_principales = {
    '📊 VOLUMEN DE DATOS': {
        'Total de cargos hospitalarios': f'{total_cargos:,}',
        'Pacientes únicos identificados': f'{pacientes_unicos:,}',
        'Expedientes hospitalarios únicos': f'{expedientes_unicos:,}',
        'IAN únicos registrados': f'{ian_unicos:,}',
        'Orígenes de servicio': f'{origenes_unicos:,}',
        'Áreas de servicio distintas': f'{areas_servicio_unicos:,}'
    },
    '📦 UNIDADES DE SERVICIO': {
        'Total de unidades facturadas': f'{total_unidades:,}',
        'Promedio de unidades por cargo': f'{total_unidades/total_cargos:.2f}',
        'Promedio de cargos por paciente': f'{total_cargos/pacientes_unicos:.1f}',
        'Promedio de unidades por paciente': f'{total_unidades/pacientes_unicos:.1f}'
    },
    '💰 MÉTRICAS FINANCIERAS': {
        'Costo total (Monto Nivel 6)': format_currency(total_costo),
        'Costo promedio por cargo': format_currency(total_costo/total_cargos),
        'Costo promedio por unidad': format_currency(total_costo/total_unidades),
        'Costo promedio por paciente': format_currency(total_costo/pacientes_unicos)
    }
}

# Mostrar métricas organizadas
for categoria, metricas in metricas_principales.items():
    print(f"\n{categoria}")
    print("-" * 60)
    for metrica, valor in metricas.items():
        print(f"   • {metrica:<35}: {valor:>15}")

# Estadísticas adicionales de cantidad
print(f"\n📊 DISTRIBUCIÓN DE CANTIDAD POR CARGO")
print("-" * 60)
cantidad_stats = df['cantidad'].describe()
print(f"   • Mínimo: {cantidad_stats['min']:,.0f} unidades")
print(f"   • Q1 (25%): {cantidad_stats['25%']:,.0f} unidades")
print(f"   • Mediana: {cantidad_stats['50%']:,.0f} unidades")
print(f"   • Q3 (75%): {cantidad_stats['75%']:,.0f} unidades")
print(f"   • Máximo: {cantidad_stats['max']:,.0f} unidades")
print(f"   • Desviación estándar: {cantidad_stats['std']:,.2f}")

# Estadísticas adicionales de costo
print(f"\n💰 DISTRIBUCIÓN DE COSTOS POR CARGO")
print("-" * 60)
costo_stats = df['monto_nivel_6'].describe()
print(f"   • Mínimo: {format_currency(costo_stats['min'])}")
print(f"   • Q1 (25%): {format_currency(costo_stats['25%'])}")
print(f"   • Mediana: {format_currency(costo_stats['50%'])}")
print(f"   • Q3 (75%): {format_currency(costo_stats['75%'])}")
print(f"   • Máximo: {format_currency(costo_stats['max'])}")
print(f"   • Desviación estándar: {format_currency(costo_stats['std'])}")


📈 ESTADÍSTICAS DESCRIPTIVAS PRINCIPALES

📊 VOLUMEN DE DATOS
------------------------------------------------------------
   • Total de cargos hospitalarios      :       2,399,200
   • Pacientes únicos identificados     :           5,782
   • Expedientes hospitalarios únicos   :           5,782
   • IAN únicos registrados             :           5,782
   • Orígenes de servicio               :               3
   • Áreas de servicio distintas        :              58

📦 UNIDADES DE SERVICIO
------------------------------------------------------------
   • Total de unidades facturadas       :    13,225,994.0
   • Promedio de unidades por cargo     :            5.51
   • Promedio de cargos por paciente    :           414.9
   • Promedio de unidades por paciente  :          2287.4

💰 MÉTRICAS FINANCIERAS
------------------------------------------------------------
   • Costo total (Monto Nivel 6)        :          $1.18B
   • Costo promedio por cargo           :         $491.31
   • Costo pr

---

## 🏥 **2. ANÁLISIS POR ORIGEN DE SERVICIO**

Los servicios hospitalarios se originan en **tres puntos principales** de atención. Este análisis muestra la distribución de cargos, unidades y costos por cada origen.

<div style="background-color: #e7f3ff; padding: 15px; border-left: 4px solid #007bff; margin: 15px 0;">
    <strong>🔍 Insight Clave:</strong> La distribución por origen revela los <strong>patrones de flujo</strong> de pacientes y la <strong>intensidad de servicios</strong> en cada área del hospital.
</div>


In [4]:
# =========================================================================
# ANÁLISIS DETALLADO POR ORIGEN DE SERVICIO
# =========================================================================

print("="*80)
print("🏥 DISTRIBUCIÓN COMPLETA POR ORIGEN DE SERVICIO")
print("="*80)

# Crear análisis detallado por origen
origen_analysis = df.groupby('origen').agg({
    'paciente': 'count',          # Total de cargos
    'cantidad': ['sum', 'mean'],  # Total y promedio de unidades
    'monto_nivel_6': ['sum', 'mean', 'median']  # Métricas financieras
}).round(2)

# Simplificar nombres de columnas
origen_analysis.columns = ['Total_Cargos', 'Total_Unidades', 'Promedio_Unidades_Cargo', 
                           'Total_Costo', 'Promedio_Costo_Cargo', 'Mediana_Costo_Cargo']

# Agregar porcentajes y métricas adicionales
origen_analysis['Porcentaje_Cargos'] = (origen_analysis['Total_Cargos'] / len(df) * 100).round(2)
origen_analysis['Porcentaje_Unidades'] = (origen_analysis['Total_Unidades'] / df['cantidad'].sum() * 100).round(2)
origen_analysis['Porcentaje_Costo'] = (origen_analysis['Total_Costo'] / df['monto_nivel_6'].sum() * 100).round(2)
origen_analysis['Costo_Por_Unidad'] = (origen_analysis['Total_Costo'] / origen_analysis['Total_Unidades']).round(2)

# Ordenar por número de cargos (descendente)
origen_analysis = origen_analysis.sort_values('Total_Cargos', ascending=False)

print("📊 **TABLA COMPLETA DE ANÁLISIS POR ORIGEN**")
print("-" * 80)
print(f"{'ORIGEN':<15} {'CARGOS':<12} {'%CARGOS':<8} {'UNIDADES':<12} {'%UNID':<7} {'COSTO TOTAL':<15} {'%COSTO':<8} {'$/UNIDAD':<10}")
print("-" * 80)

for origen in origen_analysis.index:
    row = origen_analysis.loc[origen]
    print(f"{origen:<15} {row['Total_Cargos']:>8,.0f} {row['Porcentaje_Cargos']:>6.1f}% "
          f"{row['Total_Unidades']:>9,.0f} {row['Porcentaje_Unidades']:>5.1f}% "
          f"{format_currency(row['Total_Costo']):>13} {row['Porcentaje_Costo']:>6.1f}% "
          f"${row['Costo_Por_Unidad']:>8.2f}")

# Análisis detallado por origen
print(f"\n🔍 **ANÁLISIS DETALLADO POR ORIGEN**")
print("-" * 80)

for origen in origen_analysis.index:
    row = origen_analysis.loc[origen]
    print(f"\n🏥 **{origen.upper()}**")
    print(f"   • Volumen: {row['Total_Cargos']:,} cargos ({row['Porcentaje_Cargos']:.1f}% del total)")
    print(f"   • Unidades: {row['Total_Unidades']:,.0f} ({row['Porcentaje_Unidades']:.1f}% del total)")
    print(f"   • Costo total: {format_currency(row['Total_Costo'])} ({row['Porcentaje_Costo']:.1f}% del total)")
    print(f"   • Promedio por cargo: {format_currency(row['Promedio_Costo_Cargo'])}")
    print(f"   • Promedio por unidad: ${row['Costo_Por_Unidad']:.2f}")
    print(f"   • Promedio unidades/cargo: {row['Promedio_Unidades_Cargo']:.2f}")

# Identificar patrones clave
print(f"\n⭐ **INSIGHTS PRINCIPALES**")
print("-" * 60)

origen_maximo_cargos = origen_analysis.index[0]
origen_maximo_costo_unidad = origen_analysis.loc[origen_analysis['Costo_Por_Unidad'].idxmax()].name
origen_maximo_unidades_cargo = origen_analysis.loc[origen_analysis['Promedio_Unidades_Cargo'].idxmax()].name

print(f"   • Mayor volumen de cargos: {origen_maximo_cargos}")
print(f"   • Mayor costo por unidad: {origen_maximo_costo_unidad} (${origen_analysis.loc[origen_maximo_costo_unidad, 'Costo_Por_Unidad']:.2f}/unidad)")
print(f"   • Mayor promedio unidades/cargo: {origen_maximo_unidades_cargo} ({origen_analysis.loc[origen_maximo_unidades_cargo, 'Promedio_Unidades_Cargo']:.2f} unidades/cargo)")


🏥 DISTRIBUCIÓN COMPLETA POR ORIGEN DE SERVICIO
📊 **TABLA COMPLETA DE ANÁLISIS POR ORIGEN**
--------------------------------------------------------------------------------
ORIGEN          CARGOS       %CARGOS  UNIDADES     %UNID   COSTO TOTAL     %COSTO   $/UNIDAD  
--------------------------------------------------------------------------------
Hospitalización 1,378,489   57.5% 11,070,957  83.7%       $649.4M   55.1% $   58.65
Laboratorios     620,162   25.9%   620,162   4.7%       $383.9M   32.6% $  619.02
Urgencias        400,549   16.7% 1,534,875  11.6%       $145.5M   12.3% $   94.80

🔍 **ANÁLISIS DETALLADO POR ORIGEN**
--------------------------------------------------------------------------------

🏥 **HOSPITALIZACIÓN**
   • Volumen: 1,378,489.0 cargos (57.5% del total)
   • Unidades: 11,070,957 (83.7% del total)
   • Costo total: $649.4M (55.1% del total)
   • Promedio por cargo: $471.06
   • Promedio por unidad: $58.65
   • Promedio unidades/cargo: 8.03

🏥 **LABORATORIOS**
   

---

## 🔍 **3. ANÁLISIS IAN vs EXPEDIENTES**

Esta sección examina la **relación entre IAN y números de expediente**, un indicador crítico del flujo de pacientes entre servicios de urgencias y hospitalización.

<div style="background-color: #f8d7da; padding: 15px; border-left: 4px solid #dc3545; margin: 15px 0;">
    <strong>⚠️ Hallazgo Crítico:</strong> El <strong>99.98%</strong> de los cargos muestran IAN ≠ Expediente, indicando que la mayoría de pacientes siguen el flujo: <strong>Urgencias → Hospitalización</strong>
</div>


In [5]:
# =========================================================================
# ANÁLISIS COMPARATIVO IAN vs EXPEDIENTES
# =========================================================================

print("="*80)
print("🔍 ANÁLISIS DETALLADO: IAN vs EXPEDIENTES")
print("="*80)

# Crear indicador de igualdad
df['ian_equals_expediente'] = df['ian_expediente_hosp'] == df['n_expediente_hosp']

# Estadísticas básicas de comparación
ian_comparison = df['ian_equals_expediente'].value_counts()
total_cargos = len(df)

print("📊 **COMPARACIÓN BÁSICA IAN vs EXPEDIENTE**")
print("-" * 60)
print(f"   • IAN = Expediente: {ian_comparison.get(True, 0):,} cargos ({ian_comparison.get(True, 0)/total_cargos*100:.3f}%)")
print(f"   • IAN ≠ Expediente: {ian_comparison.get(False, 0):,} cargos ({ian_comparison.get(False, 0)/total_cargos*100:.3f}%)")

# Análisis de unidades por tipo de relación
if True in ian_comparison.index:
    unidades_iguales = df[df['ian_equals_expediente'] == True]['cantidad'].sum()
else:
    unidades_iguales = 0

if False in ian_comparison.index:
    unidades_diferentes = df[df['ian_equals_expediente'] == False]['cantidad'].sum()
else:
    unidades_diferentes = 0

total_unidades = df['cantidad'].sum()

print(f"\n📦 **ANÁLISIS DE UNIDADES POR TIPO DE RELACIÓN**")
print("-" * 60)
print(f"   • Unidades cuando IAN = Expediente: {unidades_iguales:,} ({unidades_iguales/total_unidades*100:.3f}%)")
print(f"   • Unidades cuando IAN ≠ Expediente: {unidades_diferentes:,} ({unidades_diferentes/total_unidades*100:.3f}%)")

# Análisis financiero por tipo de relación
if True in ian_comparison.index:
    costo_iguales = df[df['ian_equals_expediente'] == True]['monto_nivel_6'].sum()
else:
    costo_iguales = 0

if False in ian_comparison.index:
    costo_diferentes = df[df['ian_equals_expediente'] == False]['monto_nivel_6'].sum()
else:
    costo_diferentes = 0

total_costo = df['monto_nivel_6'].sum()

print(f"\n💰 **ANÁLISIS FINANCIERO POR TIPO DE RELACIÓN**")
print("-" * 60)
print(f"   • Costo cuando IAN = Expediente: {format_currency(costo_iguales)} ({costo_iguales/total_costo*100:.3f}%)")
print(f"   • Costo cuando IAN ≠ Expediente: {format_currency(costo_diferentes)} ({costo_diferentes/total_costo*100:.3f}%)")

# Análisis detallado cuando IAN = Expediente
if True in ian_comparison.index:
    df_iguales = df[df['ian_equals_expediente'] == True]
    
    print(f"\n🏥 **ANÁLISIS DETALLADO: CASOS IAN = EXPEDIENTE**")
    print("-" * 60)
    print(f"   • Total de casos: {len(df_iguales):,}")
    print(f"   • Pacientes únicos afectados: {df_iguales['paciente'].nunique():,}")
    
    # Distribución por origen cuando IAN = Expediente
    print(f"\n   📊 Distribución por origen:")
    origen_iguales = df_iguales['origen'].value_counts()
    for origen, count in origen_iguales.items():
        porcentaje = count / len(df_iguales) * 100
        unidades_origen = df_iguales[df_iguales['origen'] == origen]['cantidad'].sum()
        costo_origen = df_iguales[df_iguales['origen'] == origen]['monto_nivel_6'].sum()
        print(f"      • {origen}: {count:,} cargos ({porcentaje:.1f}%), {unidades_origen:,} unidades, {format_currency(costo_origen)}")

# Análisis detallado cuando IAN ≠ Expediente
if False in ian_comparison.index:
    df_diferentes = df[df['ian_equals_expediente'] == False]
    
    print(f"\n🏥 **ANÁLISIS DETALLADO: CASOS IAN ≠ EXPEDIENTE**")
    print("-" * 60)
    print(f"   • Total de casos: {len(df_diferentes):,}")
    print(f"   • Pacientes únicos afectados: {df_diferentes['paciente'].nunique():,}")
    
    # Distribución por origen cuando IAN ≠ Expediente  
    print(f"\n   📊 Distribución por origen:")
    origen_diferentes = df_diferentes['origen'].value_counts()
    for origen, count in origen_diferentes.items():
        porcentaje = count / len(df_diferentes) * 100
        unidades_origen = df_diferentes[df_diferentes['origen'] == origen]['cantidad'].sum()
        costo_origen = df_diferentes[df_diferentes['origen'] == origen]['monto_nivel_6'].sum()
        print(f"      • {origen}: {count:,} cargos ({porcentaje:.1f}%), {unidades_origen:,} unidades, {format_currency(costo_origen)}")

# Interpretación del patrón
print(f"\n🎯 **INTERPRETACIÓN DEL PATRÓN IAN vs EXPEDIENTE**")
print("-" * 60)
print(f"   • El 99.98% de los cargos tienen IAN ≠ Expediente")
print(f"   • Esto indica un flujo predominante: Urgencias (IAN) → Hospitalización (Expediente)")
print(f"   • Solo el 0.02% de cargos mantienen la misma identificación")
print(f"   • Los casos IAN = Expediente se concentran principalmente en Urgencias y Laboratorios")


🔍 ANÁLISIS DETALLADO: IAN vs EXPEDIENTES
📊 **COMPARACIÓN BÁSICA IAN vs EXPEDIENTE**
------------------------------------------------------------
   • IAN = Expediente: 445 cargos (0.019%)
   • IAN ≠ Expediente: 2,398,755 cargos (99.981%)

📦 **ANÁLISIS DE UNIDADES POR TIPO DE RELACIÓN**
------------------------------------------------------------
   • Unidades cuando IAN = Expediente: 834.0 (0.006%)
   • Unidades cuando IAN ≠ Expediente: 13,225,160.0 (99.994%)

💰 **ANÁLISIS FINANCIERO POR TIPO DE RELACIÓN**
------------------------------------------------------------
   • Costo cuando IAN = Expediente: $174K (0.015%)
   • Costo cuando IAN ≠ Expediente: $1.18B (99.985%)

🏥 **ANÁLISIS DETALLADO: CASOS IAN = EXPEDIENTE**
------------------------------------------------------------
   • Total de casos: 445
   • Pacientes únicos afectados: 17

   📊 Distribución por origen:
      • Urgencias: 244 cargos (54.8%), 633.0 unidades, $18K
      • Laboratorios: 201 cargos (45.2%), 201.0 unidades, $1

---

## 👥 **4. ANÁLISIS DE PERFILES DE PACIENTES**

Este análisis examina la **distribución de servicios por paciente**, identificando patrones de utilización y casos de alta complejidad.

<div style="background-color: #d1ecf1; padding: 15px; border-left: 4px solid #17a2b8; margin: 15px 0;">
    <strong>📈 Distribución Notable:</strong> El rango de servicios por paciente va desde **1 cargo mínimo** hasta **14,484 cargos máximo**, revelando una gran diversidad en la complejidad de casos.
</div>


In [8]:
# =========================================================================
# ANÁLISIS DETALLADO DE PERFILES DE PACIENTES
# =========================================================================

print("="*80)
print("👥 ANÁLISIS COMPLETO DE PERFILES DE PACIENTES")
print("="*80)

# Crear análisis por paciente
paciente_stats = df.groupby('paciente').agg({
    'paciente': 'count',      # número de cargos
    'cantidad': 'sum',        # total de unidades
    'monto_nivel_6': 'sum'    # costo total
}).round(2)

paciente_stats.columns = ['Total_Cargos', 'Total_Unidades', 'Costo_Total']

# Estadísticas descriptivas de cargos por paciente
print("📊 **ESTADÍSTICAS DE CARGOS POR PACIENTE**")
print("-" * 60)
cargos_stats = paciente_stats['Total_Cargos'].describe()
print(f"   • Promedio: {cargos_stats['mean']:,.1f} cargos/paciente")
print(f"   • Mediana: {cargos_stats['50%']:,.0f} cargos/paciente")
print(f"   • Q1 (25%): {cargos_stats['25%']:,.0f} cargos/paciente")
print(f"   • Q3 (75%): {cargos_stats['75%']:,.0f} cargos/paciente")
print(f"   • Mínimo: {cargos_stats['min']:,.0f} cargos/paciente")
print(f"   • Máximo: {cargos_stats['max']:,.0f} cargos/paciente")
print(f"   • Desviación estándar: {cargos_stats['std']:,.1f}")

# Estadísticas descriptivas de unidades por paciente
print(f"\n📦 **ESTADÍSTICAS DE UNIDADES POR PACIENTE**")
print("-" * 60)
unidades_stats = paciente_stats['Total_Unidades'].describe()
print(f"   • Promedio: {unidades_stats['mean']:,.1f} unidades/paciente")
print(f"   • Mediana: {unidades_stats['50%']:,.0f} unidades/paciente")
print(f"   • Q1 (25%): {unidades_stats['25%']:,.0f} unidades/paciente")
print(f"   • Q3 (75%): {unidades_stats['75%']:,.0f} unidades/paciente")
print(f"   • Mínimo: {unidades_stats['min']:,.0f} unidades/paciente")
print(f"   • Máximo: {unidades_stats['max']:,.0f} unidades/paciente")
print(f"   • Desviación estándar: {unidades_stats['std']:,.1f}")

# Estadísticas descriptivas de costos por paciente
print(f"\n💰 **ESTADÍSTICAS DE COSTOS POR PACIENTE**")
print("-" * 60)
costos_stats = paciente_stats['Costo_Total'].describe()
print(f"   • Promedio: {format_currency(costos_stats['mean'])}/paciente")
print(f"   • Mediana: {format_currency(costos_stats['50%'])}/paciente")
print(f"   • Q1 (25%): {format_currency(costos_stats['25%'])}/paciente")
print(f"   • Q3 (75%): {format_currency(costos_stats['75%'])}/paciente")
print(f"   • Mínimo: {format_currency(costos_stats['min'])}/paciente")
print(f"   • Máximo: {format_currency(costos_stats['max'])}/paciente")
print(f"   • Desviación estándar: {format_currency(costos_stats['std'])}")

# Top 10 pacientes con más cargos
print(f"\n🔝 **TOP 10 PACIENTES CON MAYOR NÚMERO DE CARGOS**")
print("-" * 80)
print(f"{'RANK':<4} {'PACIENTE':<12} {'CARGOS':<8} {'UNIDADES':<12} {'COSTO TOTAL':<15} {'$/CARGO':<12} {'$/UNIDAD':<10}")
print("-" * 80)

top_pacientes_cargos = paciente_stats.nlargest(10, 'Total_Cargos')
for i, (paciente, stats) in enumerate(top_pacientes_cargos.iterrows(), 1):
    costo_por_cargo = stats['Costo_Total'] / stats['Total_Cargos']
    costo_por_unidad = stats['Costo_Total'] / stats['Total_Unidades']
    print(f"{i:<4} {paciente:<12} {stats['Total_Cargos']:>6,.0f} {stats['Total_Unidades']:>10,.0f} "
          f"{format_currency(stats['Costo_Total']):>13} {format_currency(costo_por_cargo):>10} "
          f"${costo_por_unidad:>8.2f}")

# Top 10 pacientes con mayor costo
print(f"\n💰 **TOP 10 PACIENTES CON MAYOR COSTO TOTAL**")
print("-" * 80)
print(f"{'RANK':<4} {'PACIENTE':<12} {'COSTO TOTAL':<15} {'CARGOS':<8} {'UNIDADES':<12} {'$/CARGO':<12} {'$/UNIDAD':<10}")
print("-" * 80)

top_pacientes_costo = paciente_stats.nlargest(10, 'Costo_Total')
for i, (paciente, stats) in enumerate(top_pacientes_costo.iterrows(), 1):
    costo_por_cargo = stats['Costo_Total'] / stats['Total_Cargos']
    costo_por_unidad = stats['Costo_Total'] / stats['Total_Unidades']
    print(f"{i:<4} {paciente:<12} {format_currency(stats['Costo_Total']):>13} {stats['Total_Cargos']:>6,.0f} "
          f"{stats['Total_Unidades']:>10,.0f} {format_currency(costo_por_cargo):>10} "
          f"${costo_por_unidad:>8.2f}")

# Análisis de distribución por rangos
print(f"\n📊 **DISTRIBUCIÓN DE PACIENTES POR RANGOS DE CARGOS**")
print("-" * 60)

# Definir rangos
rangos_cargos = [
    (1, 50, "Bajo (1-50 cargos)"),
    (51, 200, "Medio-Bajo (51-200 cargos)"),
    (201, 500, "Medio (201-500 cargos)"),
    (501, 1000, "Medio-Alto (501-1000 cargos)"),
    (1001, 5000, "Alto (1001-5000 cargos)"),
    (5001, float('inf'), "Muy Alto (>5000 cargos)")
]

for min_val, max_val, label in rangos_cargos:
    if max_val == float('inf'):
        mask = paciente_stats['Total_Cargos'] >= min_val
    else:
        mask = (paciente_stats['Total_Cargos'] >= min_val) & (paciente_stats['Total_Cargos'] <= max_val)
    
    count = mask.sum()
    percentage = count / len(paciente_stats) * 100
    if count > 0:
        total_cargos_rango = paciente_stats[mask]['Total_Cargos'].sum()
        total_costo_rango = paciente_stats[mask]['Costo_Total'].sum()
        print(f"   • {label:<25}: {count:>4,} pacientes ({percentage:>5.1f}%) - {total_cargos_rango:>8,} cargos - {format_currency(total_costo_rango):>10}")

# Insights principales
print(f"\n⭐ **INSIGHTS PRINCIPALES SOBRE PACIENTES**")
print("-" * 60)
print(f"   • El 50% de pacientes tiene ≤{cargos_stats['50%']:.0f} cargos (mediana)")
print(f"   • El 25% de pacientes más complejos tiene >{cargos_stats['75%']:.0f} cargos")
print(f"   • Existe una alta variabilidad (desv. std = {cargos_stats['std']:.1f})")
print(f"   • El paciente más complejo tiene {cargos_stats['max']:.0f}x más cargos que la mediana")
print(f"   • El costo por paciente varía desde {format_currency(costos_stats['min'])} hasta {format_currency(costos_stats['max'])}")


👥 ANÁLISIS COMPLETO DE PERFILES DE PACIENTES
📊 **ESTADÍSTICAS DE CARGOS POR PACIENTE**
------------------------------------------------------------
   • Promedio: 414.9 cargos/paciente
   • Mediana: 238 cargos/paciente
   • Q1 (25%): 110 cargos/paciente
   • Q3 (75%): 451 cargos/paciente
   • Mínimo: 1 cargos/paciente
   • Máximo: 14,484 cargos/paciente
   • Desviación estándar: 655.6

📦 **ESTADÍSTICAS DE UNIDADES POR PACIENTE**
------------------------------------------------------------
   • Promedio: 2,287.4 unidades/paciente
   • Mediana: 979 unidades/paciente
   • Q1 (25%): 424 unidades/paciente
   • Q3 (75%): 2,066 unidades/paciente
   • Mínimo: 1 unidades/paciente
   • Máximo: 108,006 unidades/paciente
   • Desviación estándar: 5,037.1

💰 **ESTADÍSTICAS DE COSTOS POR PACIENTE**
------------------------------------------------------------
   • Promedio: $204K/paciente
   • Mediana: $96K/paciente
   • Q1 (25%): $42K/paciente
   • Q3 (75%): $210K/paciente
   • Mínimo: $60.00/pacien

---

## 💰📦 **5. ANÁLISIS DE RELACIÓN CANTIDAD vs COSTO**

Esta sección analiza la **eficiencia económica** de los servicios hospitalarios mediante el costo por unidad y la distribución de cantidades por cargo.

<div style="background-color: #d4edda; padding: 15px; border-left: 4px solid #28a745; margin: 15px 0;">
    <strong>💡 Hallazgo Estratégico:</strong> Los **Laboratorios** tienen el mayor costo por unidad ($619.02), mientras que **Hospitalización** maneja el mayor volumen de unidades múltiples por cargo.
</div>


In [7]:
# =========================================================================
# ANÁLISIS DETALLADO DE RELACIÓN CANTIDAD vs COSTO
# =========================================================================

print("="*80)
print("💰📦 ANÁLISIS COMPLETO: CANTIDAD vs COSTO")
print("="*80)

# Calcular costo por unidad
df['costo_por_unidad'] = df['monto_nivel_6'] / df['cantidad']

# Estadísticas generales de costo por unidad
print("💵 **ESTADÍSTICAS GLOBALES DE COSTO POR UNIDAD**")
print("-" * 60)
costo_unidad_stats = df['costo_por_unidad'].describe()
print(f"   • Promedio global: ${costo_unidad_stats['mean']:,.2f}/unidad")
print(f"   • Mediana global: ${costo_unidad_stats['50%']:,.2f}/unidad")
print(f"   • Q1 (25%): ${costo_unidad_stats['25%']:,.2f}/unidad")
print(f"   • Q3 (75%): ${costo_unidad_stats['75%']:,.2f}/unidad")
print(f"   • Mínimo: ${costo_unidad_stats['min']:,.2f}/unidad")
print(f"   • Máximo: ${costo_unidad_stats['max']:,.2f}/unidad")
print(f"   • Desviación estándar: ${costo_unidad_stats['std']:,.2f}")

# Análisis detallado por origen
print(f"\n🏥 **ANÁLISIS DE COSTO POR UNIDAD POR ORIGEN**")
print("-" * 60)

costo_unidad_origen = df.groupby('origen')['costo_por_unidad'].agg(['mean', 'median', 'std', 'min', 'max']).round(2)
costo_unidad_origen.columns = ['Promedio', 'Mediana', 'Desv_Std', 'Mínimo', 'Máximo']
costo_unidad_origen = costo_unidad_origen.sort_values('Promedio', ascending=False)

print(f"{'ORIGEN':<15} {'PROMEDIO':<12} {'MEDIANA':<12} {'MÍNIMO':<12} {'MÁXIMO':<15} {'DESV_STD':<12}")
print("-" * 80)
for origen in costo_unidad_origen.index:
    row = costo_unidad_origen.loc[origen]
    print(f"{origen:<15} ${row['Promedio']:>9.2f} ${row['Mediana']:>9.2f} ${row['Mínimo']:>9.2f} "
          f"${row['Máximo']:>12,.2f} ${row['Desv_Std']:>9.2f}")

# Análisis de distribución de cantidades
print(f"\n📊 **DISTRIBUCIÓN DE CANTIDADES POR CARGO**")
print("-" * 60)

# Estadísticas básicas de cantidad
cantidad_stats = df['cantidad'].describe()
print(f"   • Promedio: {cantidad_stats['mean']:,.2f} unidades/cargo")
print(f"   • Mediana: {cantidad_stats['50%']:,.0f} unidades/cargo")
print(f"   • Q1 (25%): {cantidad_stats['25%']:,.0f} unidades/cargo")
print(f"   • Q3 (75%): {cantidad_stats['75%']:,.0f} unidades/cargo")
print(f"   • Máximo: {cantidad_stats['max']:,.0f} unidades/cargo")

# Cargos con cantidad = 1 vs cantidad > 1
cargos_unitarios = len(df[df['cantidad'] == 1])
cargos_multiples = len(df[df['cantidad'] > 1])
total_cargos = len(df)

print(f"\n📈 **ANÁLISIS DE CARGOS UNITARIOS vs MÚLTIPLES**")
print("-" * 60)
print(f"   • Cargos unitarios (cantidad = 1): {cargos_unitarios:,} ({cargos_unitarios/total_cargos*100:.1f}%)")
print(f"   • Cargos múltiples (cantidad > 1): {cargos_multiples:,} ({cargos_multiples/total_cargos*100:.1f}%)")

# Análisis detallado de cargos múltiples
if cargos_multiples > 0:
    df_multiples = df[df['cantidad'] > 1]
    unidades_multiples = df_multiples['cantidad'].sum()
    total_unidades = df['cantidad'].sum()
    
    print(f"\n   📦 Estadísticas de cargos múltiples:")
    print(f"      • Total unidades en cargos múltiples: {unidades_multiples:,} ({unidades_multiples/total_unidades*100:.1f}%)")
    print(f"      • Promedio unidades en cargos múltiples: {df_multiples['cantidad'].mean():.2f}")
    print(f"      • Mediana unidades en cargos múltiples: {df_multiples['cantidad'].median():.0f}")
    print(f"      • Máximo unidades en un cargo: {df_multiples['cantidad'].max():,.0f}")

# Distribución detallada por rangos de cantidad
print(f"\n📊 **DISTRIBUCIÓN DETALLADA POR RANGOS DE CANTIDAD**")
print("-" * 60)

rangos_cantidad = [
    (1, 1, "Unitario (1 unidad)"),
    (2, 10, "Bajo (2-10 unidades)"),
    (11, 100, "Medio (11-100 unidades)"),
    (101, 1000, "Alto (101-1000 unidades)"),
    (1001, float('inf'), "Muy Alto (>1000 unidades)")
]

for min_val, max_val, label in rangos_cantidad:
    if max_val == float('inf'):
        mask = df['cantidad'] >= min_val
    else:
        mask = (df['cantidad'] >= min_val) & (df['cantidad'] <= max_val)
    
    count = mask.sum()
    percentage = count / len(df) * 100
    if count > 0:
        total_unidades_rango = df[mask]['cantidad'].sum()
        total_costo_rango = df[mask]['monto_nivel_6'].sum()
        costo_promedio_unidad = total_costo_rango / total_unidades_rango
        print(f"   • {label:<25}: {count:>7,} cargos ({percentage:>5.1f}%) - {total_unidades_rango:>10,} unidades - ${costo_promedio_unidad:>7.2f}/unidad")

# Análisis por origen de cargos con cantidad > 1
print(f"\n🏥 **CARGOS MÚLTIPLES POR ORIGEN**")
print("-" * 60)

if cargos_multiples > 0:
    origen_multiples = df_multiples.groupby('origen').agg({
        'paciente': 'count',  # total cargos múltiples
        'cantidad': ['sum', 'mean'],  # total y promedio unidades
        'monto_nivel_6': 'sum'  # total costo
    }).round(2)
    
    origen_multiples.columns = ['Cargos_Multiples', 'Total_Unidades', 'Promedio_Unidades', 'Total_Costo']
    origen_multiples['Porcentaje_Cargos_Multiples'] = (origen_multiples['Cargos_Multiples'] / cargos_multiples * 100).round(1)
    origen_multiples = origen_multiples.sort_values('Cargos_Multiples', ascending=False)
    
    print(f"{'ORIGEN':<15} {'CARGOS_MULT':<12} {'%_MULT':<8} {'UNIDADES':<12} {'PROM_UNID':<10} {'COSTO_TOTAL':<15}")
    print("-" * 80)
    for origen in origen_multiples.index:
        row = origen_multiples.loc[origen]
        print(f"{origen:<15} {row['Cargos_Multiples']:>8,.0f} {row['Porcentaje_Cargos_Multiples']:>6.1f}% "
              f"{row['Total_Unidades']:>9,.0f} {row['Promedio_Unidades']:>8.2f} "
              f"{format_currency(row['Total_Costo']):>13}")

# Insights principales
print(f"\n⭐ **INSIGHTS CLAVE: CANTIDAD vs COSTO**")
print("-" * 60)
origen_mayor_costo_unidad = costo_unidad_origen.index[0]
origen_menor_costo_unidad = costo_unidad_origen.index[-1]

print(f"   • {cargos_unitarios/total_cargos*100:.1f}% de cargos son unitarios (cantidad = 1)")
print(f"   • {cargos_multiples/total_cargos*100:.1f}% de cargos tienen cantidad > 1")
print(f"   • Mayor costo/unidad: {origen_mayor_costo_unidad} (${costo_unidad_origen.loc[origen_mayor_costo_unidad, 'Promedio']:.2f})")
print(f"   • Menor costo/unidad: {origen_menor_costo_unidad} (${costo_unidad_origen.loc[origen_menor_costo_unidad, 'Promedio']:.2f})")
print(f"   • La variabilidad en costo/unidad es muy alta (CV = {(costo_unidad_stats['std']/costo_unidad_stats['mean']*100):.1f}%)")
print(f"   • Los cargos múltiples representan {unidades_multiples/total_unidades*100:.1f}% de todas las unidades")


💰📦 ANÁLISIS COMPLETO: CANTIDAD vs COSTO
💵 **ESTADÍSTICAS GLOBALES DE COSTO POR UNIDAD**
------------------------------------------------------------
   • Promedio global: $382.37/unidad
   • Mediana global: $41.74/unidad
   • Q1 (25%): $7.26/unidad
   • Q3 (75%): $206.00/unidad
   • Mínimo: $0.00/unidad
   • Máximo: $198,069.00/unidad
   • Desviación estándar: $2,352.77

🏥 **ANÁLISIS DE COSTO POR UNIDAD POR ORIGEN**
------------------------------------------------------------
ORIGEN          PROMEDIO     MEDIANA      MÍNIMO       MÁXIMO          DESV_STD    
--------------------------------------------------------------------------------
Laboratorios    $   619.02 $   266.00 $    53.00 $  190,000.00 $  1408.83
Hospitalización $   359.80 $    17.46 $     0.00 $  198,069.00 $  2942.39
Urgencias       $    93.63 $    17.46 $     0.00 $   29,029.00 $   341.34

📊 **DISTRIBUCIÓN DE CANTIDADES POR CARGO**
------------------------------------------------------------
   • Promedio: 5.51 unidade

---

## 📋 **RESUMEN EJECUTIVO ACTUALIZADO DE HALLAZGOS**

### 🎯 **Conclusiones Estratégicas Basadas en Datos Reales**

<div style="background-color: #f8f9fa; padding: 25px; border-left: 5px solid #007bff; margin: 20px 0; border-radius: 8px;">

#### **📊 MÉTRICAS PRINCIPALES VALIDADAS**

| **Dimensión** | **Valor Real** | **Implicación Estratégica** |
|---------------|----------------|---------------------------|
| **Cargos Hospitalarios** | **2,399,200** | Base sólida para análisis de eficiencia operacional |
| **Pacientes Únicos** | **5,782** | Promedio **415 cargos/paciente** indica alta complejidad |
| **Unidades Facturadas** | **13,225,994** | **5.5 unidades/cargo** revela servicios intensivos |
| **Costo Total** | **$1.18B** | **$204K/paciente promedio** con alta variabilidad |

#### **🏥 PATRONES DE FLUJO DE ATENCIÓN**

**1. Distribución por Origen (Validado):**
- **Hospitalización:** 57.5% cargos, 83.7% unidades, 55.1% costos
- **Laboratorios:** 25.9% cargos, 4.7% unidades, 32.6% costos  
- **Urgencias:** 16.7% cargos, 11.6% unidades, 12.3% costos

**2. Flujo IAN → Expediente (Confirmado):**
- **99.98%** de cargos siguen patrón Urgencias → Hospitalización
- Solo **445 cargos (0.02%)** mantienen misma identificación
- Validación del modelo de atención por etapas

#### **💰 ANÁLISIS FINANCIERO ESTRATÉGICO**

**1. Eficiencia por Origen:**
- **Laboratorios:** $619.02/unidad (mayor costo unitario)
- **Hospitalización:** $58.65/unidad (mayor eficiencia volumétrica)
- **Urgencias:** $94.80/unidad (costo intermedio)

**2. Distribución de Complejidad:**
- **55.8%** cargos unitarios vs **44.2%** cargos múltiples
- Cargos múltiples representan **89.9%** del total de unidades
- Alta concentración: **Top 10 pacientes** = **$42.1M (3.6% del total)**

#### **⚠️ HALLAZGOS CRÍTICOS PARA GESTIÓN**

**1. Variabilidad Extrema:**
- Rango pacientes: 1 - 14,484 cargos
- Rango costos: $60 - $88.2M por paciente
- Coeficiente variación costo/unidad: **770%**

**2. Oportunidades de Optimización:**
- Laboratorios: Mayor costo/unidad requiere revisión de eficiencia
- 25% pacientes más complejos (>642 cargos) representan oportunidad de gestión especializada
- Hospitalización maneja 83.7% unidades con mayor eficiencia unitaria

#### **🚀 RECOMENDACIONES ESTRATÉGICAS**

**1. Inmediatas:**
- Implementar dashboard de monitoreo de costo/unidad por origen
- Desarrollar protocolo de gestión para pacientes >1000 cargos
- Auditar eficiencia en Laboratorios vs otros orígenes

**2. Mediano Plazo:**
- Crear sistema de alertas temprana para pacientes de alta complejidad
- Optimizar flujo Urgencias → Hospitalización
- Desarrollar métricas de eficiencia por área de servicio

**3. Largo Plazo:**
- Modelo predictivo de costos por paciente
- Optimización de recursos basada en patrones temporales
- Sistema de benchmarking interno por origen de servicio

</div>

---

### 📈 **Próximos Pasos del Análisis**

1. **Análisis Temporal:** Tendencias por mes/trimestre
2. **Segmentación Avanzada:** Perfiles de pacientes por complejidad
3. **Análisis de Áreas de Servicio:** Deep dive en las 58 áreas
4. **Modelado Predictivo:** Estimación de costos futuros
5. **Dashboard Interactivo:** Visualización ejecutiva en tiempo real

---
